# UTS

## Install library

In [1]:
!pip install gensim
!pip install tqdm
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
import pandas as pd #pandas
import numpy as np #numpy
from tqdm.auto import tqdm
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
stop_words = set(stopwords.words('indonesian'))

In [10]:
df = pd.read_csv('/content/drive/MyDrive/prosaindata/tugas/pta_fix.csv')

In [11]:
df

,stem,label
0,sistem informasi akademik siakad sistem inform...,rpl
1,berjalannya koneksi jaringan komput lancar gan...,rpl
2,web server perangkat lunak server berfungsimen...,rpl
3,seir perkembangan teknolog didunia muncul tekn...,komputasi
4,gerak pekerja game memiliki genr rt realtim st...,komputasi
...,...,...
591,investasi saham memiliki resiko kerugian dikar...,komputasi
592,inform retriev ir pengambilan informasi tersim...,pba
593,klasifikasi citra prose pengelompokan piksel c...,komputasi
594,identifikasi atribut pejalan kaki salah peneli...,komputasi


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [13]:
sentences = [word_tokenize(kata) for kata in tqdm(df.stem)]

  0%|          | 0/596 [00:00<?, ?it/s]

## Train Word2Vec Model

In [14]:
model_word2vec = Word2Vec(sentences, vector_size=128, window=5, min_count=3, workers=4, sg=0, hs=0, epochs=1000)

### Save Word2Vec Model

In [15]:
folder = 'w2v/'
path = '/content/drive/MyDrive/prosaindata/prosaindata/' + folder 
os.makedirs(path, exist_ok=True)

In [16]:
model_word2vec.save(path+'text_klasifikasi.w2v')

## Encoding

In [17]:
from gensim.models import Word2Vec

In [18]:
w2v = Word2Vec.load(path+'text_klasifikasi.w2v').wv

In [19]:
def sent_vector(sentence, w2v_model, stopword):
  vecs = [w2v_model[kata.lower()] for kata in (sentence) if kata in w2v_model]
  sent_vec = np.mean(vecs, axis=0)
  return sent_vec

def norm_sent_vector(sentence, w2v_model, stopword):
  vecs = [w2v_model[kata.lower()] for kata in (sentence) if kata not in stopword]
  norm_vecs = [vec / np.linalg.norm(vec) for vec in vecs if np.linalg.norm(vec) > 0]
  sent_vec = np.mean(norm_vecs, axis=0)
  return sent_vec

In [20]:
vecs = [sent_vector(kata, w2v, stop_words) for kata in sentences]
vecs = np.array(vecs)
vecs.shape

(596, 128)

## Dataset split

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
X = vecs
y = df.label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
X_train.shape, X_test.shape

((476, 128), (120, 128))

## Training Model

In [24]:
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [25]:
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

### Naive Bayes

In [26]:
nb = GaussianNB()
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)
acc_nb = accuracy_score(y_test, y_pred_nb)
precision_nb = precision_score(y_test, y_pred_nb, average='macro')
recall_nb = recall_score(y_test, y_pred_nb, average='macro')
f1_nb = f1_score(y_test, y_pred_nb, average='macro')

In [27]:
print('Accuracy of naive Bayes classifier: {:.2f}'.format(acc_nb))
print('Precision of naive Bayes classifier: {:.2f}'.format(precision_nb))

Accuracy of naive Bayes classifier: 0.78
Precision of naive Bayes classifier: 0.70


### KNN

In [28]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_pca, y_train)
y_pred_knn = knn.predict(X_test_pca)
acc_knn = accuracy_score(y_test, y_pred_knn)

In [29]:
print('Accuracy of k-NN classifier: {:.2f}'.format(acc_knn))
print('Precision of k-NN classifier: {:.2f}'.format(acc_knn))

Accuracy of k-NN classifier: 0.75
Precision of k-NN classifier: 0.75
